In [ ]:
import pandas as pd
# !pip install pandas_datareader
from pandas_datareader import data as web
from datetime import datetime
from datetime import date
import investpy as inv


In [88]:
#ler massa de dados
dreitr = pd.read_csv('DADOS/itr_cia_aberta_DRE_con_2011-2021')
dredfp = pd.read_csv('DADOS/dfp_cia_aberta_DRE_con_2011-2021')

# relação de empresas
empresas = dreitr[['DENOM_CIA','CD_CVM','CNPJ_CIA']].drop_duplicates().set_index('CD_CVM')
# Relação de contas
# contas = empresa[['CD_CONTA','DS_CONTA']].drop_duplicates().set_index('CD_CONTA')
# '3.99.01.01'


# Manter uma relação de datas padrão
data_ini = '01/01/2011'
data_fim = '01/01/2022'



In [115]:
def CalculoLPA(codigo_cvm,conta_cvm='3.99.01.01'):

    #Selecao de empresas nos arquivos principais
    empresa = dreitr[dreitr['CD_CVM']==codigo_cvm]
    empresadfp = dredfp[dredfp['CD_CVM']==codigo_cvm]

    #LPA do ITR
    lpa = empresa[empresa['CD_CONTA'] == conta_cvm]
    lpa = lpa[lpa['ORDEM_EXERC'] == 'ÚLTIMO']
    lpa['DT_INI_EXERC'] = pd.to_datetime(lpa['DT_INI_EXERC'])
    lpa['DT_FIM_EXERC'] = pd.to_datetime(lpa['DT_FIM_EXERC'])
    lpa['Timedelta'] = pd.to_timedelta(lpa['DT_FIM_EXERC']-lpa['DT_INI_EXERC'], unit='D', errors='raise') #calcular o delta dos registros para localizar os registros apenas de trimestres separados e não de acumulado do ano
    lpa = lpa[lpa['Timedelta'] < pd.Timedelta(100, unit='D')] #filtrar somente registros de trimestres em separado
    lpa = lpa[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')

    #LPA do DFP
    lpadfp = empresadfp[empresadfp['ORDEM_EXERC'] == 'ÚLTIMO']
    lpadfp = lpadfp[lpadfp['CD_CONTA'] == conta_cvm]
    lpadfp['DT_INI_EXERC'] = pd.to_datetime(lpadfp['DT_INI_EXERC'])
    lpadfp['DT_FIM_EXERC'] = pd.to_datetime(lpadfp['DT_FIM_EXERC'])
    lpadfp = lpadfp[['DT_FIM_EXERC','VL_CONTA']].set_index('DT_FIM_EXERC')
    lpa['ano'] = lpa.index.year #testar remover linha
    lpadfp['ano'] = lpadfp.index.year


    #Obter lpa somente do quarto tri
    lpaano = lpa.groupby('ano').sum()
    lpaano.rename(columns={'VL_CONTA':'VL_CONTA_AC_3Q'}, inplace=True)
    lpadfpano = lpadfp.groupby('ano').sum()
    lpadfpano.rename(columns={'VL_CONTA':'VL_CONTA_AC_4Q'}, inplace=True)

    lpa4q = lpaano.join(lpadfpano)
    lpa4q['VL_CONTA_SO_4Q'] = lpa4q['VL_CONTA_AC_4Q'] - lpa4q['VL_CONTA_AC_3Q']
    lpa4q.reset_index(inplace=True)
    lpa4q['date'] = lpa4q['ano'].astype(str)+'-12-31'
    lpa4q = lpa4q[['date','ano','VL_CONTA_SO_4Q']].set_index('date')
    lpa4q.rename(columns={'VL_CONTA_SO_4Q':'VL_CONTA'}, inplace=True)

    #Consolidar dataframe com LPAs por trimestre
    lpa_trimestral = pd.concat([lpa, lpa4q]).drop('ano',axis=1)
    lpa_trimestral.index = pd.to_datetime(lpa_trimestral.index)
    lpa_trimestral.sort_index(inplace=True)

    lpa_trimestral['ultimos 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).sum()
    lpa_trimestral['media 4q'] = lpa_trimestral['VL_CONTA'].rolling(4, min_periods=4).mean()

    lpa_trimestral['CD_CVM'] = codigo_cvm
    lpa_trimestral['CD_CONTA'] = conta_cvm


    # Obter ticker b3

    ticker = pd.read_excel('AUXILIAR\Empresas_Cadastro_Full.xlsx')
    tickerPN = ticker[ticker['CD_CVM'] == 7617]['PN'].iloc[0]
    tickerON = ticker[ticker['CD_CVM'] == 7617]['ON'].iloc[0]


    if conta_cvm == '3.99.01.01': #ON
        lpa_trimestral['TP_TICKER'] = tickerON  #talvez daria para levar o nome do ticker somente no arquivo salvo, nao precisaria ter uma serie repetindo
        ticker = tickerON
    elif conta_cvm == '3.99.01.02': #PN
        lpa_trimestral['TP_TICKER'] = tickerPN
        ticker = tickerPN
    else:
        lpa_trimestral['TP_TICKER'] = ''

    
    lpa_trimestral.index.name = 'Date'

    # Obter cotações do periodo


    
    
    cotacoes = pd.DataFrame(inv.get_stock_historical_data(stock=ticker,
                                        country='brazil',
                                        from_date=data_ini,
                                        to_date=data_fim)['Close'])


    # cotacoes = cotacoes.join(lpa_trimestral)

    df_output = pd.date_range(start=data_ini,end=data_fim)
    df_output = pd.DataFrame(df_output, columns=['Date']).set_index('Date')
    df_output = df_output.join(cotacoes).join(lpa_trimestral)

    return df_output.ffill()


In [116]:
CalculoLPA(7617,'3.99.01.02')

,Close,VL_CONTA,ultimos 4q,media 4q,CD_CVM,CD_CONTA,TP_TICKER
Date,,,,,,,
2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-03,3.80,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,3.85,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,3.90,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2021-12-28,9.08,0.28091,1.40003,0.350008,7617.0,3.99.01.02,ITSA4
2021-12-29,9.03,0.28091,1.40003,0.350008,7617.0,3.99.01.02,ITSA4
2021-12-30,8.93,0.28091,1.40003,0.350008,7617.0,3.99.01.02,ITSA4


In [ ]:
# !pip freeze requirements.txt




In [ ]:
# gerar todas as planilhas de LPA

# for empr in empresas.index:
#     df = CalculoLPA(empr)
#     df.to_excel('LPA/lpa_'+str(empr)+'.xlsx')

# Salva Relacao de empresas

# empresas.to_excel('AUXILIAR/Empresas_CD_CVM_e_CNPJ.xlsx')

In [66]:
cotxx = CalculoLPA(7617,'3.99.01.02')[0]
lpaxx = CalculoLPA(7617,'3.99.01.02')[1]

In [100]:

dates = pd.date_range(start=data_ini,end=data_fim)
dates = pd.DataFrame(dates, columns=['Date']).set_index('Date')
dates.join(cotxx).join(lpaxx)

,Close,VL_CONTA,ultimos 4q,media 4q,CD_CVM,CD_CONTA,TP_TICKER
Date,,,,,,,
2011-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-03,3.80,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,3.85,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,3.90,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2021-12-28,9.08,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-29,9.03,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-30,8.93,NaN,NaN,NaN,NaN,NaN,NaN
